In [1]:
import numpy as np
import pandas as pd


In [3]:
# read json file
df = pd.read_json("src/mockData.json")
df

,patientId,priorityRank,sepsisScore,hazardRate,lastVitalTime,temp,hr,location,trends
0,31408129,1,72,0.902,8,101.3,128,ICU North,"[{'time': '3h ago', 'temp': 101.9, 'heartRate'..."
1,31391746,2,91,0.887,14,100.6,122,ICU South,"[{'time': '3h ago', 'temp': 100.2, 'heartRate'..."
2,39788549,3,68,0.919,22,102.1,130,Stepdown East,"[{'time': '3h ago', 'temp': 101.5, 'heartRate'..."
3,30998534,4,83,0.884,11,99.8,118,Stepdown West,"[{'time': '3h ago', 'temp': 100.1, 'heartRate'..."
4,31367180,5,97,0.777,35,100.2,110,Telemetry A,"[{'time': '3h ago', 'temp': 100.8, 'heartRate'..."
5,34897912,6,74,0.704,17,98.9,105,Telemetry B,"[{'time': '3h ago', 'temp': 99.1, 'heartRate':..."
6,30154746,7,88,0.688,42,99.5,102,Observation 3,"[{'time': '3h ago', 'temp': 99.9, 'heartRate':..."
7,32858107,8,69,0.559,28,101.7,116,Observation 1,"[{'time': '3h ago', 'temp': 101.3, 'heartRate'..."
8,37560316,9,95,0.545,56,99.1,96,Med-Surg North,"[{'time': '3h ago', 'temp': 99.4, 'heartRate':..."
9,39747582,10,79,0.508,63,98.4,90,Med-Surg South,"[{'time': '3h ago', 'temp': 98.7, 'heartRate':..."


In [6]:
# load reference csv file
ref_df = pd.read_csv("../../data/MIMIC-ED/event_level_training_data.csv")
ref_df

,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,rhythm_flag,is_white,...,arrival_transport_WALK IN,lactate,wbc,time_since_adm,gsn_16599.0,gsn_43952.0,gsn_4490.0,gsn_66419.0,gsn_61716.0,is_sepsis
0,37459204,98.1,79.0,18.0,98.0,126.0,72.0,0.0,0,1,...,0,1.6,7.8,0.016667,1,0,0,0,0,0
1,37459204,102.9,92.0,18.0,94.0,123.0,53.0,0.0,0,1,...,0,1.6,7.8,0.083333,1,0,0,0,0,1
2,37459204,102.9,92.0,18.0,94.0,123.0,53.0,0.0,0,1,...,0,1.6,7.8,0.150000,0,0,0,0,0,1
3,37459204,104.2,83.0,22.0,93.0,111.0,48.0,0.0,0,1,...,0,3.2,7.8,0.450000,0,0,0,0,0,1
4,37459204,104.2,83.0,22.0,93.0,111.0,48.0,0.0,0,1,...,0,3.2,15.6,1.083333,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96551,39316677,98.5,118.0,23.0,97.0,117.0,82.0,0.0,0,0,...,1,1.8,7.7,10.650000,0,0,0,0,0,1
96552,39316677,98.5,118.0,23.0,97.0,117.0,82.0,0.0,0,0,...,1,1.8,7.7,11.033333,0,0,0,0,0,1
96553,39316677,98.5,130.0,20.0,99.0,132.0,100.0,0.0,0,0,...,1,1.8,7.7,11.133333,0,0,0,0,0,1
96554,39316677,98.0,124.0,19.0,100.0,121.0,73.0,0.0,0,0,...,1,1.8,7.7,11.516667,0,0,0,0,0,1


In [14]:
# extract out ids
all_ids = df['patientId'].tolist()

In [18]:
cols_needed = ['stay_id','temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'rhythm_flag', 'is_white', 'is_black', 'is_asian', 'is_hispanic', 'is_other_race', 'gender_F', 'gender_M', 'arrival_transport_AMBULANCE', 'arrival_transport_HELICOPTER', 'arrival_transport_OTHER', 'arrival_transport_UNKNOWN', 'arrival_transport_WALK IN', 'lactate', 'wbc', 'time_since_adm', 'gsn_16599.0', 'gsn_43952.0', 'gsn_4490.0', 'gsn_66419.0', 'gsn_61716.0']
example_id = 37459204

extracted_rows = ref_df[ref_df['stay_id'].isin(all_ids)][cols_needed]

# only keep last row for each stay_id
extracted_rows = extracted_rows.groupby('stay_id').tail(1)
extracted_rows

,stay_id,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,rhythm_flag,is_white,...,arrival_transport_UNKNOWN,arrival_transport_WALK IN,lactate,wbc,time_since_adm,gsn_16599.0,gsn_43952.0,gsn_4490.0,gsn_66419.0,gsn_61716.0
3220,30998534,102.8,123.0,20.0,99.0,114.0,78.0,0.0,0,0,...,1,0,2.9,8.0,3.950000,0,0,0,0,0
5805,34897912,98.1,79.0,18.0,98.0,126.0,72.0,0.0,0,1,...,0,0,0.9,14.4,4.083333,0,0,0,0,0
11369,32858107,97.8,105.0,19.0,96.0,94.0,80.0,0.0,0,1,...,0,0,1.7,8.1,12.283333,0,0,0,0,0
14452,39747582,99.0,99.0,20.0,98.0,105.0,69.0,7.0,0,1,...,0,0,1.0,9.9,10.650000,0,0,0,0,0
17419,37560316,99.0,92.0,17.0,95.0,157.0,66.0,0.0,0,1,...,0,1,1.6,1.0,11.766667,0,0,0,0,0
26076,30154746,99.7,88.0,16.0,98.0,111.0,61.0,3.0,0,1,...,0,1,1.8,16.9,11.666667,0,1,0,0,0
50679,31391746,98.0,67.0,16.0,100.0,109.0,66.0,0.0,0,1,...,0,0,1.6,7.8,2.333333,0,0,0,0,0
62647,31367180,98.1,88.0,16.0,100.0,142.0,70.0,0.0,0,0,...,0,1,1.6,11.1,3.000000,0,0,0,0,0
78812,31408129,97.8,91.0,28.0,95.0,108.0,62.0,0.0,0,0,...,0,0,1.7,17.7,14.166667,0,0,0,0,0
80085,39788549,97.9,73.0,17.0,99.0,141.0,88.0,5.0,0,1,...,0,1,1.6,7.8,5.033333,0,0,0,0,0


In [19]:
# rename heartrate to hr and temperature to temp
extracted_rows = extracted_rows.rename(columns={"heartrate": "hr", "temperature": "temp", "stay_id": "patientId"})
extracted_rows

,patientId,temp,hr,resprate,o2sat,sbp,dbp,pain,rhythm_flag,is_white,...,arrival_transport_UNKNOWN,arrival_transport_WALK IN,lactate,wbc,time_since_adm,gsn_16599.0,gsn_43952.0,gsn_4490.0,gsn_66419.0,gsn_61716.0
3220,30998534,102.8,123.0,20.0,99.0,114.0,78.0,0.0,0,0,...,1,0,2.9,8.0,3.950000,0,0,0,0,0
5805,34897912,98.1,79.0,18.0,98.0,126.0,72.0,0.0,0,1,...,0,0,0.9,14.4,4.083333,0,0,0,0,0
11369,32858107,97.8,105.0,19.0,96.0,94.0,80.0,0.0,0,1,...,0,0,1.7,8.1,12.283333,0,0,0,0,0
14452,39747582,99.0,99.0,20.0,98.0,105.0,69.0,7.0,0,1,...,0,0,1.0,9.9,10.650000,0,0,0,0,0
17419,37560316,99.0,92.0,17.0,95.0,157.0,66.0,0.0,0,1,...,0,1,1.6,1.0,11.766667,0,0,0,0,0
26076,30154746,99.7,88.0,16.0,98.0,111.0,61.0,3.0,0,1,...,0,1,1.8,16.9,11.666667,0,1,0,0,0
50679,31391746,98.0,67.0,16.0,100.0,109.0,66.0,0.0,0,1,...,0,0,1.6,7.8,2.333333,0,0,0,0,0
62647,31367180,98.1,88.0,16.0,100.0,142.0,70.0,0.0,0,0,...,0,1,1.6,11.1,3.000000,0,0,0,0,0
78812,31408129,97.8,91.0,28.0,95.0,108.0,62.0,0.0,0,0,...,0,0,1.7,17.7,14.166667,0,0,0,0,0
80085,39788549,97.9,73.0,17.0,99.0,141.0,88.0,5.0,0,1,...,0,1,1.6,7.8,5.033333,0,0,0,0,0


In [32]:
# join on patientId
final_df = pd.merge(df, extracted_rows, on='patientId', how='inner')
final_df = final_df.drop(["temp_x", "hr_x"], axis=1).rename(columns={"temp_y": "temp", "hr_y": "hr"})
# reorder so that trends is at the end
cols = final_df.columns.tolist()
cols.remove('trends')
cols.append('trends')
final_df = final_df[cols]
final_df

,patientId,priorityRank,sepsisScore,hazardRate,lastVitalTime,location,temp,hr,resprate,o2sat,...,arrival_transport_WALK IN,lactate,wbc,time_since_adm,gsn_16599.0,gsn_43952.0,gsn_4490.0,gsn_66419.0,gsn_61716.0,trends
0,31408129,1,72,0.902,8,ICU North,97.8,91.0,28.0,95.0,...,0,1.7,17.7,14.166667,0,0,0,0,0,"[{'time': '3h ago', 'temp': 101.9, 'heartRate'..."
1,31391746,2,91,0.887,14,ICU South,98.0,67.0,16.0,100.0,...,0,1.6,7.8,2.333333,0,0,0,0,0,"[{'time': '3h ago', 'temp': 100.2, 'heartRate'..."
2,39788549,3,68,0.919,22,Stepdown East,97.9,73.0,17.0,99.0,...,1,1.6,7.8,5.033333,0,0,0,0,0,"[{'time': '3h ago', 'temp': 101.5, 'heartRate'..."
3,30998534,4,83,0.884,11,Stepdown West,102.8,123.0,20.0,99.0,...,0,2.9,8.0,3.950000,0,0,0,0,0,"[{'time': '3h ago', 'temp': 100.1, 'heartRate'..."
4,31367180,5,97,0.777,35,Telemetry A,98.1,88.0,16.0,100.0,...,1,1.6,11.1,3.000000,0,0,0,0,0,"[{'time': '3h ago', 'temp': 100.8, 'heartRate'..."
5,34897912,6,74,0.704,17,Telemetry B,98.1,79.0,18.0,98.0,...,0,0.9,14.4,4.083333,0,0,0,0,0,"[{'time': '3h ago', 'temp': 99.1, 'heartRate':..."
6,30154746,7,88,0.688,42,Observation 3,99.7,88.0,16.0,98.0,...,1,1.8,16.9,11.666667,0,1,0,0,0,"[{'time': '3h ago', 'temp': 99.9, 'heartRate':..."
7,32858107,8,69,0.559,28,Observation 1,97.8,105.0,19.0,96.0,...,0,1.7,8.1,12.283333,0,0,0,0,0,"[{'time': '3h ago', 'temp': 101.3, 'heartRate'..."
8,37560316,9,95,0.545,56,Med-Surg North,99.0,92.0,17.0,95.0,...,1,1.6,1.0,11.766667,0,0,0,0,0,"[{'time': '3h ago', 'temp': 99.4, 'heartRate':..."
9,39747582,10,79,0.508,63,Med-Surg South,99.0,99.0,20.0,98.0,...,0,1.0,9.9,10.650000,0,0,0,0,0,"[{'time': '3h ago', 'temp': 98.7, 'heartRate':..."


In [33]:
# output to json make
final_df.to_json("src/mockData.json", orient="records", lines=False, indent=1)